### Conexão e DB

In [13]:
# Rewrite Database
from DatabaseDAO import DatabaseDAO

CONNECTION = {
    'host': 'localhost',
    'port': 5432,
    'database': 'postgres',
    'user': 'postgres',
    'password': '999590188aA'
}

conn = DatabaseDAO(**CONNECTION)
conn.DEBUG = False

conn.execute_query('SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE datname = %s', ['meumenu'])
#conn.execute_query('DROP DATABASE meumenu;')
conn.execute_query('CREATE DATABASE meumenu;')

CONNECTION = {
    'host': 'localhost',
    'port': 5432,
    'database': 'meumenu',
    'user': 'postgres',
    'password': '999590188aA'
}

conn = DatabaseDAO(**CONNECTION)
conn.DEBUG = False

with open('structure.sql') as f:
    sql = f.read()
    conn.execute_query(sql)

conn.DEBUG = True

In [1]:
# Just Connect
from DatabaseDAO import DatabaseDAO

CONNECTION = {
    'host': 'localhost',
    'port': 5432,
    'database': 'meumenu',
    'user': 'postgres',
    'password': '999590188aA'
}

conn = DatabaseDAO(**CONNECTION)
conn.DEBUG = False

In [2]:
from Store import Store, create_store, store_login

### Testes

In [ ]:
# Create New Store
my_store = create_store(
        conn,
        name='Lojão do Pão',
        phone='+5585996108737',
        email='airton.neto@delfosim.com',
        password='droped123',
        specialtys=['Pães', 'Queijos'],
        adress_country='Brazil',
        adress_city='Fortaleza',
        adress_borough='Joaquim Távora',
        adress_street='Rua Visconde do Rio Branco 2955, APTO 202 A'
)

In [ ]:
# Instantiate class Store
store_id = my_store['id']
store = Store(conn, my_store['id'])

In [ ]:
# Categories
category = store.create_category('Pães', 'O melhor pão da região!')
category2 = store.create_category('Queijos', 'O melhor queijo da região!')
store.delete_category(category2['id'])
store.categories()

[{'id': 1,
  'name': 'Pães',
  'description': 'O melhor pão da região!',
  'store_id': 1,
  'store_name': 'Lojão do Pão'}]

In [ ]:
# Specialtys
store.update_specialtys(['Pães', 'Queijos', 'Bolos'])
store.specialtys()

[{'specialtys': ['Pães', 'Queijos', 'Bolos']}]

In [ ]:
# Schedules
schedule = store.add_schedule(1, 3, '13:00', '17:00')
schedule2 = store.add_schedule(5, 7, '05:00', '17:00')
store.delete_schedule(schedule2['id'])
store.schedules()

[{'id': 1,
  'store_id': 1,
  'dow_start': 1,
  'dow_end': 3,
  'opens_at': '13:00:00',
  'closes_at': '17:00:00'}]

In [ ]:
# Products
with open('pao_carioca.jpeg', 'rb') as f:
    photo = f.read()
product = store.create_product(
    name='Pão Carioquinha', 
    description='O melhor pão da cidade!', 
    price=0.5, 
    photo=photo, 
    category_id=category['id']
)
with open('pao_carteira.jpg', 'rb') as f:
    photo = f.read()
product2 = store.create_product(
    name='Pão Carteira', 
    description='Pãozinho Artesanal Fresquinho para sua família!', 
    price=0.5, 
    photo=photo, 
    category_id=category['id']
)
store.delete_product(product2['id'])
store.products()

### API

In [5]:
from flask import Flask, request, make_response
import json

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Welcome to MeuMenu API!</p>"

# Create New Store
@app.route("/store/sign", methods = ["POST"])
def app_create_store():
    data = request.get_json()
    return create_store(
        conn,
        data['name'],
        data['phone'],
        data['email'],
        data['password'],
        data['specialtys'],
        data['adress_country'],
        data['adress_city'],
        data['adress_borough'],
        data['adress_street']
    )

@app.route("/store/login", methods = ["GET"])
def app_store_information():
    params = request.args
    data = store_login(conn, **params)
    if not data.empty:
        return data.to_dict(orient = 'records')[0]
    else:
        return make_response("Non Authorized", 401)

@app.route("/store/<store_id>/categories", methods = ["GET", "POST", "DELETE"])
def app_categories(store_id):

    store = Store(conn, store_id)
    if request.method == 'GET':
        params = request.args
        return {'response': store.categories()}
    
    elif request.method == 'POST':
        data = request.get_json()
        return store.create_category(**data)
        
    elif request.method == 'DELETE':
        data = request.get_json()
        return store.delete_category(**data)

import base64
import io

@app.route("/store/<store_id>/products", methods = ["GET", "POST", "DELETE"])
def app_products(store_id):

    store = Store(conn, store_id)
    if request.method == 'GET':
        params = request.args
        return {'response': store.products()}
    
    elif request.method == 'POST':
        data = request.get_json()
        
        im_b64 = data.pop('photo')
        img_bytes = base64.b64decode(im_b64.encode('utf-8'))
        img = io.BytesIO(img_bytes)
        data['photo'] = img_bytes
        
        return store.create_product(**data)
        
    elif request.method == 'DELETE':
        data = request.get_json()
        return store.delete_product(**data)

@app.route("/store/<store_id>/specialtys", methods = ["GET", "PUT"])
def app_specialtys(store_id):

    store = Store(conn, store_id)
    if request.method == 'GET':
        params = request.args
        return json.dumps(store.specialtys()[0])
    
    elif request.method == 'PUT':
        data = request.get_json()
        return store.update_specialtys(**data)

@app.route("/store/<store_id>/schedules", methods = ["GET", "POST", "DELETE"])
def app_schedules(store_id):

    store = Store(conn, store_id)
    if request.method == 'GET':
        params = request.args
        return {'response': store.schedules()}
    
    elif request.method == 'POST':
        data = request.get_json()
        return store.add_schedule(**data)
        
    elif request.method == 'DELETE':
        data = request.get_json()
        return store.delete_schedule(**data)
    
@app.route("/store/<store_id>/schedules/delete", methods = ["POST"])
def app_schedulesDelete(store_id):
    store = Store(conn, store_id)
    data = request.get_json()
    return store.delete_schedule(**data)

app.run("0.0.0.0", port = 5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.103:5000/ (Press CTRL+C to quit)
192.168.1.103 - - [02/Aug/2021 17:53:59] "GET /store/login/?email=airton.neto@delfosim.com&password=droped123 HTTP/1.1" 404 -
192.168.1.103 - - [02/Aug/2021 17:55:36] "GET /store/login/?email=airton.neto@delfosim.com&password=droped123 HTTP/1.1" 404 -


primeiro if


192.168.1.103 - - [02/Aug/2021 17:55:47] "GET /store/login?email=airton.neto@delfosim.com&password=droped123 HTTP/1.1" 200 -


   id          name           phone                     email   password  \
0   1  Lojão do Pão  +5585996108737  airton.neto@delfosim.com  droped123   

  specialtys adress_country adress_city  adress_borough  \
0      [Pao]         Brazil   Fortaleza  Joaquim Távora   

                                 adress_street  
0  Rua Visconde do Rio Branco 2955, APTO 202 A  


[2021-08-02 18:00:30,973] ERROR in app: Exception on /store/login [GET]
Traceback (most recent call last):
  File "C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\flask\app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\flask\app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\flask\app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\flask\app.py", line 1499, in dispatch_request
    return 

primeiro if
   id          name           phone                     email   password  \
0   1  Lojão do Pão  +5585996108737  airton.neto@delfosim.com  droped123   

  specialtys adress_country adress_city  adress_borough  \
0      [Pao]         Brazil   Fortaleza  Joaquim Távora   

                                 adress_street  
0  Rua Visconde do Rio Branco 2955, APTO 202 A  
